In [ ]:
source(paste0(dirname(dirname(dirname(getwd()))),'/map.r'))
source(paste0(HELP_DIR, "shortcuts.r"))
source(paste0(HELP_DIR, "helpers.r"))

# Read prepped cohorts data

- Read in prepared data and output

In [ ]:
base <- fread(paste0(SHARE_DIR, "2_run_marginal_output.csv")) 

In [ ]:
cohorts <- fread(paste0(SHARE_DIR, "top_mechanisms.csv"))

In [ ]:
treatment_mechanism_map <- 
fread(paste0(SHARE_DIR, "treatment_mechanism_map.csv")) %>% 
 mu(derived_treatmentName = gsub( "##","/", derived_treatmentName),
    derived_treatmentMechanism = gsub( "##","/", derived_treatmentMechanism))

In [ ]:
go <- fread(paste0(SHARE_DIR, "fisher_base.csv"))

# Let's go
- Add fields

In [ ]:
low_responder_threshold <- .05
pval_threshold <- .1

In [ ]:
orderer <- 
c('Anti-PD-1',
  'Immunotherapy',
  'Chemotherapy',
  'Anti-AR',
  ' ')

In [ ]:
lets_go <- 
base %>% 
 fi(total_patients >= 30, events >= 6, non_responders >= 15, responders >= 15, surv_se <= 5) %>% 
 mu(p_fdr_fisher = p.adjust(fisher_pval, method = "fdr"), 
    p_fdr_fisher_by = p.adjust(fisher_pval, method = "BY"),
    p_fdr_surv = p.adjust(surv_pval, method = "fdr"),
    p_fdr_surv_by = p.adjust(surv_pval, method = "BY"),
    or = ifelse(or == "Inf", exp(5), or), 
    surv_high = surv_est + 1.96*surv_se, surv_low = surv_est - 1.96*surv_se,
    prob_response = e_r/events,
    low_responder = (prob_response <= low_responder_threshold),
    sig_fisher = (p_fdr_fisher <= pval_threshold), sig_pfs = (p_fdr_surv <= pval_threshold),
    light_highlight = ((low_responder) & (fisher_pval < .02) & (surv_pval < .05)),
    highlight_fisher = ((low_responder) & (sig_fisher) & (surv_pval < .05)), 
    highlight_pfs = ((low_responder) & (sig_pfs) & (fisher_pval < .05)),
    dark_highlight = ((highlight_fisher ) & (highlight_pfs)),
    Odds = ifelse(direction == "Response", "Better", "Worse"),
    pan = grepl("Pan-Cancer", cohortGo)) %>% 
  rw() %>% mu( derived_treatmentName = str_split_fixed( cohortGo	, " / ", n = 2)[2]) %>% ug() %>% 
  lj(cohorts %>% se(cohortGo, group), by = "cohortGo") %>% 
  lj( treatment_mechanism_map , by = "derived_treatmentName") %>% 
  mu( derived_treatmentMechanism = ifelse(is.na(derived_treatmentMechanism), derived_treatmentName, derived_treatmentMechanism),
      treatment = derived_treatmentName, 
      mechanism = gsub(" ## ", "/", derived_treatmentMechanism),
      Treatment = factor(ifelse(highlight_fisher, mechanism, " "), levels = orderer)  ) %>% 
  se(-direction) %>% 
  mu(
  gp = factor(
    case_when(
     dark_highlight ~ "Both Signficant",
     highlight_fisher ~ "Fisher Signficant",  
     highlight_pfs ~ "PFS Signficant",   
     light_highlight ~ "Both Significant / Unadjusted",
     TRUE ~ "Rest"), 
       levels = c("Both Signficant","Fisher Signficant","PFS Signficant", "Both Significant / Unadjusted","Rest")))

# Examples

- Initial examples

In [ ]:
tmp <- 
lets_go %>% 
 fi(highlight_fisher | highlight_pfs | (grepl("driver", feature) & light_highlight) | (grepl("driver", feature) & light_highlight)) %>% 
 fi( !((cohortGo == "Skin Melanoma / Anti-PD-1") & (feature == "drivers_pathway_IMMUNE_EVASION_gt0"))) %>%
 gb(cohortGo) %>%
 mu(rk = row_number(desc(e_nr))) %>%
 tm(cohortGo, feature, fisher_pval, surv_pval, e_nr, e_r, select_example = TRUE, rk) %>% 
 ar(cohortGo, fisher_pval)

In [ ]:
remove_examples <- c("signature_log_ID2_gt75", "lilac_hla_cn_B44_gt0", "rna_geneset_KEGG_CYSTEINE_AND_METHIONINE_METABOLISM_gt75")

In [ ]:
examples_base <- tmp %>% fi(rk == 1, !feature %in% remove_examples) %>% tm(cohortGo, feature, selected_example = TRUE)

In [ ]:
examples <- 
examples_base %>%
bind_rows(
lets_go %>% 
 fi(cohortGo == "Prostate / Anti-AR", feature == "rna_geneset_KEGG_TGF_BETA_SIGNALING_PATHWAY_gt50") %>%
 tm(cohortGo, feature, selected_example = TRUE))

In [ ]:
lets_go <- lets_go %>% lj(examples, by = c("cohortGo", "feature"))

# Nice Labels

- Need to process names for downstream plots

In [ ]:
highlights <- examples %>% pu(feature)

In [ ]:
replacements <- 
c("_" = " ", 
  "rna geneset " = "RNA", 
  "gene set" = "",
  "HALLMARK" = "", 
  "KEGG" = "",
  "gt0" = "",
  "gt75" = "Very High",
  "gt50" = "High",
  "gt25" = "Moderate/High",
  "lt75" = "Low/Moderate",
  "lt50" = "Low",
  "lt25" = "Very Low",
  "gt" = "> ",
  "hotspot KRAS position 25398284" = "KRAS G12D hotspot",
  "purity tmbPerMb lt6" = "TMB per MB < 6",
  "neo ct" = "RNA Neoantigens",
  "TGF BETA SIGNALING PATHWAY" = "TGFB",
  "SIGNALING " = "",
  " AND " = "/")

# Loop over names and replace
s1 <- highlights
for (pattern in names(replacements)) {
  s1 <- gsub(pattern, replacements[pattern], s1)
}

In [ ]:
updates <- 
c("Kras" = "KRAS", "Rna" = "RNA", 
  "Tim" = "TIM", "Tgfb" = "TGFB", 
  "G12d" = "G12D", 
  "Erbb" = "ERBB", 
  "Wnt" = "WNT",
  "Nrf2" = "NRF2",  
  "Ddr" = "DDR",
  "Abcb" = "ABCB",
  "Complement/Coagulation Cascades" = "Complement/Coagulation",
  "Hippo" = "HIPPO",
  "B2m" = "B2M",
  "Caf Lrrc15 Fibroblasts" = "CAF",
  "Taurine/Hypotaurine" = "Taurine",
  "Log Id2" = "SBS2")

In [ ]:
ready <- str_to_title(s1)
for (pattern in names(updates)) {
  ready <- gsub(pattern, updates[pattern], ready)
}

In [ ]:
name_map <- as.list(trimws(ready))
names(name_map) <- highlights

In [ ]:
next_step <- 
lets_go %>% 
 rw() %>% 
 mu(clean_name = ifelse(feature %in% names(name_map), name_map[[feature]], feature)) %>% 
 ug() %>% 
 mu(example = paste0(clean_name, "\n", cohortGo))

# Send processed data

In [ ]:
saveRDS(next_step, paste0(SHARE_DIR, "3_ready_marginal.Rds"))